In [ ]:
from spotify_object import spotify
import base64
import requests
import datetime
import json
import random
from time import sleep
from urllib.parse import urlencode
import pandas as pd
import numpy as np

In [ ]:
"""I CULTIVATED A PLAYLIST WITH 100 OF MY FAV SONGS TO TRAIN OFF OF, this is just a foundation"""

#Let's see if this works
#Need to log into account and get playlist account
#CODE IS USER ID
playlists = spotify.get_user_playlists('20y4cuyjnc87e3vxc2qqzl69q')

# playlist_name = input("What playlist would you like to model off of?: ")

In [ ]:
#scrub through the playlists that match the input of their favorite songs
#lets create a list to then pick which playlist you want 

playlist_list = []
for i in range(100):
    try:
        item = playlists['items'][i]['name']
        playlist_list.append(f"{i+1}. {item}")
    except IndexError:
        continue  

In [ ]:
#print out the list of playlists to choose from 
print("Please Select a playlist to model off of!!")
print("Try to Select one with more than 50 songs for best results!!")
print("")
print("PLAYLISTS:")
for i in playlist_list:
    print(i)
    
#make an input for the list
selection = int(input("What number playlist do you want?:  ")) - 1

In [ ]:
#now to gather the results from that playlist
_id = playlists['items'][selection]['id']
playlist_items = spotify.get_playlist_items(_id)
#this is for looping purposes later down the line 
total_tracks = playlists['items'][selection]['tracks']['total']

In [ ]:
#creating a dataframe from the songs in the playlist
d = []
for i in range(total_tracks):
    try:
        d.append(
            {
                'track': playlist_items['items'][i]['track']['name'],
                'artist': playlist_items['items'][i]['track']['artists'][0]['name'],
                'id': playlist_items['items'][i]['track']['id'],
                'preview': playlist_items['items'][i]['track']['preview_url'],
                'popularity': playlist_items['items'][i]['track']['popularity']

            }
    )
    except KeyError:
        continue
df = pd.DataFrame(d)

In [ ]:
#assigning the song ids for features 
song_ids = list(df['id'])
#iterating for the features 
f = []
for track in song_ids:
    features = spotify.get_features(track)
    f.append(features)
df2 = pd.DataFrame(f)

In [ ]:
#merging the dataframes together
to_model = pd.merge(df,df2, on='id')

In [ ]:
"""GATHERING MORE SONGS!!!"""

In [ ]:
# ok its time to get more data so we can tell the computer what not to look for 
"""found this playlist on the web with over 2000 songs"""
big_playlist = spotify.get_playlist_items('5fMCrRnSy4TauAmM36zrIP')
#this will only get me the first 100 songs, so I must use the next link

#just creating the total_tracks variable for looping
total_tracks = big_playlist['total']

In [ ]:
#need to take next from each playlist then collect of the items from there
#Spotify has max limit of 100 results, so need to go through offset in order to use it
def get_playlist_tracks(playlist_id):
    results = spotify.get_playlist_items(playlist_id)
    tracks = results['items']
    while results['next']:
        results = spotify.url_search(results['next'])
        tracks.extend(results['items'])
    return tracks

In [ ]:
#using the function defined above
big_playlist = get_playlist_tracks('5fMCrRnSy4TauAmM36zrIP')

In [ ]:
#repeating the process from above in mapping a dictionary, but this time with the big ol' playlist 
b = []
for i in range(total_tracks):
    try:
        b.append(
            {
                'track': big_playlist[i]['track']['name'],
                'artist': big_playlist[i]['track']['artists'][0]['name'],
                'id': big_playlist[i]['track']['id'],
                'preview': big_playlist[i]['track']['preview_url'],
                'popularity': big_playlist[i]['track']['popularity']

            }
    )
    except KeyError:
        continue
df = pd.DataFrame(b)

In [ ]:
song_ids = list(df['id'])
#iterating for the features 
f = []
for track in song_ids:
    features = spotify.get_features(track)
    f.append(features)
df2 = pd.DataFrame(f)
#merging the dfs together
curated = pd.merge(df,df2, on = 'id')

In [ ]:
#lets read out the curated playlists for further analysis
curated.to_csv('curated.csv')

In [ ]:
#creating a target variable
to_model['like'] = 1
curated['like'] = None

In [ ]:
#sampling out the dataframe for input
new = curated.sample(n=500,random_state=123)


In [ ]:
#reseting the index
new.reset_index(drop = True, inplace = True)

In [ ]:
#do user input
like = []
for i in range(len(new)):
    answer = input(f"{new['track'][i]} by {new['artist'][i]}: ")
    like.append(answer)
    
    
    

In [ ]:
"""THIS SECTION IS ACTIVLEY GOING OUT AND GATHERING PLAYLISTS FROM SPOTIFY/WE ARE NOT THERE YET!"""

In [ ]:
#search for specific genre with name in playlist

"""This only looks for playlists with the genre in the title for instance, there will be more spotify curated playlists"""
genre = input("What Genre would you be looking for today my liege?: ")
search = spotify.search(query = f"{genre}", search_type = "playlist", limit = 10)  


In [ ]:
#we want to create a repository of the 10 most popular playlists
#do another loopty loop 
playlist_list = []
for i in range(10):
    id = search['playlists']['items'][i]['id']
    playlist_list.append((id))

In [ ]:
#{specific_genre:[{playlist_id:[items]}]}

In [ ]:
#so now we take the song id and create a dataframe with those then search for features to model the recommendation of fof

"""Create a big ol loopty loop"""

#the playlist dictionary that stores the song_id
playlist_dict = {}
song_ids = []
#looping through the 10 playlists that are included in the genre specified
for i in range(10):
    id = search['playlists']['items'][i]['id']
    items = spotify.get_playlist_items(id)
    #the gathering of song ids within the playlist
    
    #creating a list to hold the playlist items
    for i in range(100):
        try:
            _id = items['items'][i]['track']['id']
            song_ids.append(_id)
        except TypeError:
            continue
        except IndexError:
            continue
    #updating the dictionary and appending it to the playlist_dict
#     to_update = {id:song_ids}
#     playlist_dict.update(to_update)

        

In [ ]:
#creating a dataframe from the songs in the list of 10 playlists
d = []
for track in song_ids:
    try:
        song = spotify.get_track(track)
        d.append(
            {
                'track': song['name'],
                'artist': song['album']['artists'][0]['name'],
                'id':  track,
                'preview': song['preview_url'],
                'image_url': song['album']['images'][0]['url'],
                'popularity': song['popularity']

            }
    )
    except KeyError:
        continue
df = pd.DataFrame(d)

In [ ]:
#iterating through the song features on tracks
f = []
for track in song_ids:
    features = spotify.get_features(track)
    f.append(features)
df2 = pd.DataFrame(f)

In [ ]:
#merging the features with the tracks on the shared id
to_model = pd.merge(df,df2, on='id')